# Parsing data from decision tree spreadsheets

In [ ]:
import xlrd
import os
import pandas as pd

In [ ]:
_PATH = '../data/decision_tree/'
DATA_FILE_NAMES = sorted(os.listdir(_PATH))
print(DATA_FILE_NAMES)

## Parsing `diagnostic_data_matrix_mar2015.xlsx`

In [ ]:
FILE_NAME = 'diagnostic_data_matrix_mar2015.xlsx'
book = xlrd.open_workbook(_PATH + FILE_NAME)

In [ ]:
print("The number of worksheets is {0}".format(book.nsheets))
print("Worksheet name(s): {0}".format(book.sheet_names()))

In [ ]:
INDEX = 1
URL_INDEX = 'http://www.ipm.ucdavis.edu'


def retrieve_problems(col_index, sh):
  name = sh.cell_value(rowx = 0, colx = col_index)
  link = sh.cell_value(rowx = 1, colx = col_index)
  macrotype = sh.cell_value(rowx = 2, colx = col_index)
  microtype = sh.cell_value(rowx = 3, colx = col_index)
  d = {1: 'vegetable', 2: 'fruit', 3: 'woody', 4: 'flower'}
  plant = d[sh.number]
  
  problem = {
      'name': name,
      'link': URL_INDEX + link,
      'macrotype': macrotype,
      'microtype': microtype,
      'plant': plant,
      
  }
  return problem

def check(to_parse):
  assert isinstance(to_parse, str) and len(to_parse) > 0
  tmp = to_parse.lower().split(':')
  assert len(tmp) == 2
  return tmp

def retrieve_part(to_parse):
  tmp = to_parse.strip()
  assert len(tmp) > 0
  return [tmp.strip()]

def retrieve_descs(to_parse):
  descs = [i.strip() for i in to_parse.split(",")]
  assert len(descs) > 0 and "" not in descs
  return descs

def parse_sheet(index = 1, debug = False):
  sh = book.sheet_by_index(index)
  print(f"{sh.name} {sh.nrows} {sh.ncols}")
  print(f"Cell A1 is {sh.cell_value(rowx=0, colx=0)}")

  START_COL = 4
  START_ROW = 8
  END_COL = sh.ncols
  END_ROW = sh.nrows
  
  df = {
      'problem_name': [],
      'problem_link': [],
      'problem_macrotype': [],
      'problem_microtype': [],
      'problem_plant': [],
      'problem_part': [],
      'problem_description': [],
      'problem_desc_link': []
  }

  for c in range(START_COL, END_COL):
    problem = retrieve_problems(c, sh)
    for r in range(START_ROW, END_ROW):
      tmp = sh.cell_value(rowx = r, colx = c)
      if tmp == '':
        continue
      desc_link = tmp
      tmp = sh.cell_value(rowx = r, colx = 0)
      parts, descs = check(tmp)
      parts = retrieve_part(parts)
      descs = retrieve_descs(descs)
      tmp = sh.cell_value(rowx = r, colx = 1)
      if tmp != '':
        tmp_part, tmp_descs = check(tmp)
        tmp_part = retrieve_part(tmp_part)
        tmp_descs = retrieve_descs(tmp_descs)
        parts.extend(tmp_part)
        descs.extend(tmp_descs)
      tmp = sh.cell_value(rowx = r, colx = 2)
      if tmp != '':
        tmp_part, tmp_descs = check(tmp)
        tmp_part = retrieve_part(tmp_part)
        tmp_descs = retrieve_descs(tmp_descs)
        parts.extend(tmp_part)
        descs.extend(tmp_descs)
      
      parts = list(set(parts))
      if debug: print(r, parts)
      assert len(parts) == 1
      descs = list(set(descs))

      df['problem_name'].append(problem['name'])
      df['problem_link'].append(problem['link'])
      df['problem_macrotype'].append(problem['macrotype'])
      df['problem_microtype'].append(problem['microtype'])
      df['problem_plant'].append(problem['plant'])
      
      df['problem_desc_link'].append(desc_link)
      df['problem_part'].append(parts[0])
      df['problem_description'].append(descs)
    
  return df

In [ ]:
df = parse_sheet(index = 1)
df = pd.DataFrame(data = df)
df.sample(10)

In [ ]:
INDEX = 2
tmp = parse_sheet(index = INDEX)
tmp = pd.DataFrame(data = tmp)

print(df.shape, tmp.shape)
df = df.append(tmp)
print(df.shape)

In [ ]:
INDEX = 3
tmp = parse_sheet(index = INDEX)
tmp = pd.DataFrame(data = tmp)

print(df.shape, tmp.shape)
df = df.append(tmp)
print(df.shape)

In [ ]:
INDEX = 4
tmp = parse_sheet(index = INDEX)
tmp = pd.DataFrame(data = tmp)

print(df.shape, tmp.shape)
df = df.append(tmp)
print(df.shape)

In [ ]:
df.sample(10)

In [ ]:
print(f'Problem macrotype:\n{df["problem_macrotype"].unique()}\n')
print(f'Problem microtype:\n{df["problem_microtype"].unique()}\n')
print(f'Problem plant:\n{df["problem_plant"].unique()}\n')
print(f'Problem part:\n{df["problem_part"].unique()}\n')
print(f'Shape:\n{df.shape}\n')
print(f'Shape of duplicated parts:\n{df[df["problem_part"].isin(["twigs/branches", "flowers/blossoms", "roots/crown", "nuts/kernels"])].shape}')


In [ ]:
rename = {
    'Invertebrates' : 'Invertebrate', 
    'Diseases'      : 'Disease'     ,
    'Invertebrate ' : 'Invertebrate'
}
df['problem_macrotype'].replace(rename, inplace = True)

rename = {'Unknown': 'Other'}
df['problem_microtype'].replace(rename, inplace = True)

df['problem_part'] = df['problem_part'].str.split('/')
df = df.explode('problem_part')

df['problem_description'] = df['problem_description'].str.join(', ').replace({'"nests"': 'nests'})

print(f'Problem macrotype:\n{df["problem_macrotype"].unique()}\n')
print(f'Problem microtype:\n{df["problem_microtype"].unique()}\n')
print(f'Problem plant:\n{df["problem_plant"].unique()}\n')
print(f'Problem part:\n{df["problem_part"].unique()}\n')
print(f'Shape:\n{df.shape}\n')

In [ ]:
df.sample(10)

## Parsing `diagnostic_data_list_mar2021.xlsx`

In [ ]:
FILE_NAME = 'diagnostic_data_list_mar2021.xlsx'
book = xlrd.open_workbook(_PATH + FILE_NAME)

In [ ]:
print("The number of worksheets is {0}".format(book.nsheets))
print("Worksheet name(s): {0}".format(book.sheet_names()))

In [ ]:
sh = book.sheet_by_index(0)
print(f"{sh.name} {sh.nrows} {sh.ncols}")
print(f"Cell A1 is {sh.cell_value(rowx=0, colx=0)}")

In [ ]:
START_ROW = 1
END_ROW = sh.nrows

plant_type = []
plant_name = []
plant_part = []
plant_damage = []
link = []
for r in range(START_ROW, END_ROW):
  tmp_plant_type = sh.cell_value(rowx = r, colx = 0)
  tmp_plant_name = sh.cell_value(rowx = r, colx = 1)
  tmp_plant_part = sh.cell_value(rowx = r, colx = 2)
  tmp_plant_damage = sh.cell_value(rowx = r, colx = 3)
  tmp_link = sh.cell_value(rowx = r, colx = 4)
  plant_type.append(tmp_plant_type)
  plant_name.append(tmp_plant_name)
  plant_part.append(tmp_plant_part)
  plant_damage.append(tmp_plant_damage)
  link.append(tmp_link)

df = pd.DataFrame(data = {'plant_type': plant_type, 
                          'plant_name': plant_name, 
                          'plant_part': plant_part, 
                          'plant_damage': plant_damage,
                          'link': link
                          })

df.sample(10)

In [ ]:
print(f'Plant type:\n{df["plant_type"].unique()}\n')
print(f'Plant part:\n{df["plant_part"].unique()}\n')
print(f'Shape:\n{df.shape}\n')
print(f'Shape of duplicated parts:\n{df[df["plant_part"].isin(["flowers/blossoms", "twigs/branches"])].shape}')

In [ ]:
pt_map = {
    'Flowers (non-woody)': 'flowers', 
    'Flowers': 'flowers',
    'Fruit trees, nuts, berries, and grapevines': 'fruits',
    'Trees and shrubs (including roses)': 'trees',
    'Vegetables, melons, and herbs': 'vegetables'
}
df['plant_type'].replace(pt_map, inplace = True)

df['plant_damage'] = df['plant_damage'].str.replace('"nests"', 'nests')

df['plant_part'] = df['plant_part'].str.split('/')
df = df.explode('plant_part')

print(f'Shape:\n{df.shape}\n')
print(f'Plant type:\n{df["plant_type"].unique()}\n')
print(f'Plant part:\n{df["plant_part"].unique()}\n')

In [ ]:
df.sample(10)